# TM10007 Assignment template

In [36]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

# General packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets as ds
import seaborn

# Preprocessing
from sklearn import model_selection

# Classifiers and kernels
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA, KernelPCA
from sklearn.kernel_approximation import RBFSampler
from sklearn.metrics.pairwise import rbf_kernel, sigmoid_kernel
from sklearn.model_selection import GridSearchCV, StratifiedKFold 
from sklearn import metrics
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.preprocessing import LabelEncoder

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [37]:
from worclipo.load_data import load_data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')
print(data)

The number of samples: 115
The number of columns: 494
                  label  PREDICT_original_sf_compactness_avg_2.5D  \
ID                                                                  
Lipo-001_0  liposarcoma                                  0.368434   
Lipo-002_0  liposarcoma                                  0.660571   
Lipo-003_0       lipoma                                  0.365191   
Lipo-004_0  liposarcoma                                  0.372210   
Lipo-005_0       lipoma                                  0.369120   
...                 ...                                       ...   
Lipo-111_0       lipoma                                  0.244813   
Lipo-112_0  liposarcoma                                  0.197353   
Lipo-113_0  liposarcoma                                  0.307562   
Lipo-114_0       lipoma                                  0.577333   
Lipo-115_0  liposarcoma                                  0.635282   

            PREDICT_original_sf_compactness_std_

In [38]:
"""Splitting the data into a test and training dataset: outer cross-validation"""
#Outer cross-validation
#Split the dataset in train and test part
data_train, data_test = model_selection.train_test_split(data, test_size=0.20)
print(f'The number of samples train: {len(data_train.index)}')
print(f'The number of columns train: {len(data_train.columns)}')
print(f'The number of samples test: {len(data_test.index)}')
print(f'The number of columns test: {len(data_test.columns)}')

The number of samples train: 92
The number of columns train: 494
The number of samples test: 23
The number of columns test: 494


In [39]:
# Outliers detecteren en vervangen
from outliers import outlier_detection
data_train, total_outliers = outlier_detection(data_train)
print("\nTotaal aantal gecapte outliers train data:", total_outliers)

data_test, total_outliers = outlier_detection(data_test)
print("\nTotaal aantal gecapte outliers test data:", total_outliers)

c:\Users\Eigenaar\Documents\TM\TM10007 Machine Learning\TM10007_ML_Project_group_9\outliers.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[26.22771825 26.22771825 26.22771825 26.22771825 26.22771825 26.22771825
 26.22771825]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[outliers,column] = np.clip(data.loc[outliers,column], mean_value - drempel * std_value, mean_value + drempel * std_value)
c:\Users\Eigenaar\Documents\TM\TM10007 Machine Learning\TM10007_ML_Project_group_9\outliers.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[33.14158187]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[outliers,column] = np.clip(data.loc[outliers,column], mean_value - drempel * std_value, mean_value + drempel * std_value)
c


Totaal aantal gecapte outliers train data: 659


c:\Users\Eigenaar\Documents\TM\TM10007 Machine Learning\TM10007_ML_Project_group_9\outliers.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[31.60544092]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[outliers,column] = np.clip(data.loc[outliers,column], mean_value - drempel * std_value, mean_value + drempel * std_value)



Totaal aantal gecapte outliers test data: 150


In [40]:
from prepro_data import processing_data_scaling
#This data is scaled and a variance and correlation threshold are applied
data_scaled, df_label, df_processed= processing_data_scaling(data_train)
print(data_scaled)

                  label         0         1         2         3         4  \
ID                                                                          
Lipo-111_0       lipoma  1.046424  1.950225  0.729093 -0.204546 -0.175615   
Lipo-084_0       lipoma  1.355212  0.102615 -0.467550 -0.329645 -0.207760   
Lipo-065_0       lipoma -0.965959 -0.582315  0.405107 -1.030710 -0.936837   
Lipo-114_0       lipoma -0.349059 -0.389721 -0.674806 -0.619719 -0.800725   
Lipo-034_0       lipoma -0.112904 -0.694500  2.701530  0.465353 -0.488316   
...                 ...       ...       ...       ...       ...       ...   
Lipo-026_0  liposarcoma -0.892029 -0.777413 -0.495750 -0.709496 -0.770561   
Lipo-091_0  liposarcoma -0.280154 -0.017238  0.134416  0.371488 -0.018799   
Lipo-005_0       lipoma  1.996356  2.538911 -0.698561 -0.269294 -0.419165   
Lipo-008_0       lipoma -0.611692 -0.360713 -0.371533 -0.936209 -0.929684   
Lipo-112_0  liposarcoma -0.002600 -0.029101 -0.755156 -0.661088 -0.647563   

In [55]:
from prepro_data import processing_data_pca
#This data is scaled, a variance and correlation threshold are applied, 
# and PCA brought back the number of features to 36
data_pca, df_label, df_processed= processing_data_pca(data_train)
print(data_pca)
print(df_label)
print(df_processed)

                  label          0         1         2         3         4  \
ID                                                                           
Lipo-111_0       lipoma   7.578773  0.850528 -1.627097  3.541338 -0.885937   
Lipo-084_0       lipoma   6.804972 -2.850512  0.888432  3.728307  0.835807   
Lipo-065_0       lipoma  -5.338789  8.412473 -1.204792  0.262620 -7.047418   
Lipo-114_0       lipoma  -2.719615 -1.376493 -1.736499  2.071009 -0.598393   
Lipo-034_0       lipoma   1.657788  5.054503  2.386510  3.179803 -0.480565   
...                 ...        ...       ...       ...       ...       ...   
Lipo-026_0  liposarcoma  -4.955020  0.722217 -2.743274  2.122968  1.167446   
Lipo-091_0  liposarcoma  -2.339074  2.088565  0.730526  0.028711  1.122469   
Lipo-005_0       lipoma   7.218561 -5.116223 -2.206003 -2.986597 -0.873260   
Lipo-008_0       lipoma -10.755804 -6.864974  2.625137  2.340066 -1.205544   
Lipo-112_0  liposarcoma   3.017558 -0.749031  3.100895  0.875523

In [ ]:
from prepro_data import processing_data_rfecv
#This data is scaled, a variance and correlation threshold are applied,
# and RFECV brought back the number of features to 61 (takes a bit longer to run)
data_rf, df_label, df_processed= processing_data_rfecv(data_train)
print(data_rf)

In [53]:
"""Support Vector Machine"""
# Split data into features and labels
X = data_pca.drop(data_pca.columns[[0, 1]], axis=1)  # Features
Y = df_label['label']  # Labels

# Set up the parameter grid for GridSearchCV
param_grid = [
    {'kernel': ['linear'], 'C': [0.1, 1, 10]},
    {'kernel': ['rbf'], 'C': [0.1, 1, 10], 'gamma': ['scale', 'auto', 0.1, 1]},
    {'kernel': ['poly'], 'C': [0.1, 1, 10], 'degree': [3, 4, 5], 'gamma': ['scale', 'auto', 0.1, 1], 'coef0': [0.0, 0.1, 0.5]},
]

# Set up the SVM classifier
svm_classifier = SVC(random_state=42, probability=True)  # Enable probability for AUC calculation

# Set up inner cross-validation for hyperparameter tuning
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Set up GridSearchCV to search for the best hyperparameters
grid_search = GridSearchCV(
    estimator=svm_classifier, 
    param_grid=param_grid, 
    cv=inner_cv, 
    n_jobs=-1, 
    verbose=1, 
    scoring=['roc_auc', 'f1'],  # Optimize for both AUC and F1-score
    refit='roc_auc'  # Select the best model based on AUC
)

# Perform the grid search on the training data to find the best hyperparameters
grid_search.fit(X, Y)

# Get the best model (with the best hyperparameters)
best_model = grid_search.best_estimator_

# Make predictions to evaluate performance
y_pred = best_model.predict(X)
y_pred_proba = best_model.predict_proba(X)[:, 1]  # Probability estimates for AUC

# Convert categorical labels to numerical labels (0 or 1)
le = LabelEncoder()
Y = le.fit_transform(Y)  # 'lipoma' -> 0, 'liposarcoma' -> 1

# Ensure predictions are also numerical
y_pred = best_model.predict(X)
y_pred = le.transform(y_pred)  # Convert categorical predictions ('lipoma', 'liposarcoma') to 0 and 1

# Calculate evaluation metrics
auc_score = roc_auc_score(Y, y_pred_proba)
f1 = f1_score(Y, y_pred)

# Print results
print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Training AUC: {auc_score:.4f}")
print(f"Training F1-score: {f1:.4f}")

Fitting 5 folds for each of 123 candidates, totalling 615 fits
Best hyperparameters: {'C': 1, 'coef0': 0.5, 'degree': 4, 'gamma': 'scale', 'kernel': 'poly'}
Training AUC: 1.0000
Training F1-score: 0.9890


c:\Users\Eigenaar\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
c:\Users\Eigenaar\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


In [65]:
"""Random Forest"""
from prepro_data import split_data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, cross_validate
from sklearn.metrics import make_scorer, f1_score, roc_auc_score
from scipy.stats import randint

# Split data into features and labels
data_selection, data_id, df_label = split_data(data_train)

rf_classifier = RandomForestClassifier(random_state=42)

# Define the outer cross-validation (for evaluating model performance)
outer_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Define the inner cross-validation (for hyperparameter tuning)
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

X = data_selection  # Features
Y = df_label      # Labels

# Define the parameter grid for GridSearchCV
param_dist = {
    'n_estimators': randint(10, 200),        # Randomly sample number of trees between 10 and 200
    'max_depth': [None, 10, 20, 30, 40],      # Randomly sample max_depth options
    'min_samples_split': randint(2, 20),     # Randomly sample minimum samples for split
    'min_samples_leaf': randint(1, 20),      # Randomly sample minimum samples for leaf
    'bootstrap': [True, False]               # Bootstrap sampling options
}

# Define scoring metrics for cross-validation
scoring = {
    'accuracy': 'accuracy',
    'f1': make_scorer(f1_score, pos_label='liposarcoma'),  # Specify 'liposarcoma' as the positive class for F1 score
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True, multi_class='ovr', average='macro')  # Multi-class ROC-AUC
}

random_search = RandomizedSearchCV(
    rf_classifier, param_distributions=param_dist, n_iter=20, cv=inner_cv, random_state=42, n_jobs=-1, verbose=2)

random_search.fit(X, Y)
print("Best Hyperparameters from RandomizedSearchCV:", random_search.best_params_)

# Perform cross-validation and get multiple metrics
cv_results = cross_validate(
    random_search, X, Y, scoring=scoring, cv=outer_cv, return_train_score=False)

# Print the evaluation metrics
print(f'Nested Cross-Validation Accuracy: {cv_results["test_accuracy"].mean()}')
print(f'Average F1 Score: {cv_results["test_f1"].mean()}')
print(f'Average ROC-AUC: {cv_results["test_roc_auc"].mean()}')





c:\Users\Eigenaar\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Users\Eigenaar\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Best Hyperparameters from RandomizedSearchCV: {'bootstrap': True, 'max_depth': 40, 'min_samples_leaf': 7, 'min_samples_split': 12, 'n_estimators': 97}
Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Users\Eigenaar\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Users\Eigenaar\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Users\Eigenaar\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Nested Cross-Validation Accuracy: 0.6207885304659498
Average F1 Score: 0.6136521136521137
Average ROC-AUC: 0.6820370370370371
